<a id='root'></a>
PyUltraLight 2.21.3 for Paper 1 
(12 May 2021)

* Y. Wang: [yourong.f.wang@auckland.ac.nz](mailto:yourong.f.wang@auckland.ac.nz)
* R. Easther
***



<a id='init'></a>
# Initialization and Program Settings

## Loading Packages

In [ ]:
###### Do not touch
MinVersion = 20

import PyUltraLight2 as PyUL

if (PyUL.S_version < MinVersion):
    raise RuntimeError("You need the latest PyULN!")

PyUL.PyULCredits()

import numpy as np

np.set_printoptions(suppress=True)

import math

import numba
import numexpr as ne
import time
import pyfftw
import os
import sys
import multiprocessing
import numpy

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec
import matplotlib.animation

import IPython
from IPython.core.display import clear_output, display, Video

from numpy import sqrt, exp, log, log10, sin, cos, tan, pi

%reload_ext autoreload
%autoreload 2

# Useful Aux Functions
ToCode = PyUL.convert
ToPhys = PyUL.convert_back
CB = PyUL.convert_between
printU = PyUL.printU

## Visualization Preferences
Initialize the colorised representations for plots and movies.

### Publication

In [ ]:
plt.style.use('default')

plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams["text.usetex"]= False
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.facecolor'] = 'w'

from matplotlib.colors import LinearSegmentedColormap
Palette = 'magma'
cyccol=['#000000', '#ffffff','#000000']
divcol=['#003262','#005b96','#ffffff','#d5a756','#B51700']
divcols=['#005b96','#ffffff','#d5a756']

CycPalette = LinearSegmentedColormap.from_list('myCyc', cyccol)

DivPalette  = LinearSegmentedColormap.from_list('myDiv', divcol)
DivPaletteS  = LinearSegmentedColormap.from_list('myDiv', divcols)

EFigSize = (20,12)
EFigSizeMono = (20,6)
VideoSize = (8,8)
FPS = 10
DPI = 72

FrameCap = 60 # Maximum Number of Frames to Animate

########################################

# BH Trajectory

Colors = np.array([[209,17,65],[0,177,89],[0,174,219],[243,119,53],[255,196,37]])/255

# NBody
ENStyle = '-.'
ENColor = '#ff6c0e' # NBody

EKColor = '#8dd9e7'
EPColor = '#fd8dc8'

# ULDM
EUStyle = '--'
EUColor = '#7f1717' # ULDM

EQColor = '#ccb603' # Kinetic 
ERColor = '#6cc25c' # Self-Interaction
EGColor = '#2c8899' # Particle Potential

# Totals
ETStyle = '-'
ETColor = '#080808'

# Auxiliary 
HelperStyle = 'k.'
EmphColor = 'black'
GeomColor = 'orange'

[**Back to the Top**](#root)
<a id='run'></a>
# Batch Simulation and Quick Analysis


In [ ]:
baseConfig = './Reusable/QDrag1_QW'

In [ ]:
batch_path = 'Batches'
ExperimentName = '/Pub1_ERS_4'

PyUL.DSManagement(batch_path)

batch_path += ExperimentName

PyUL.DSManagement(batch_path)

In [ ]:
# Currently supported scan parameters: 

#Density: ULDM Initial Density. Inherits Config density_unit.
#Resolution: N
#TM_M: BH Mass. Inherits Config m_mass_unit.
#TM_v: BH Speed. Inherits Config m_speed_unit.
#U_v: QuantumWind Speed. Inherits Config s_speed_unit.
#Step_Factor: Multiplicative Factor on Timestep Length.
#Scaling: Multiplicative Factor on Domain Size
#Plummer_Radius: Multiples of Mesh Spacing.

ScanParams = ['Step_Factor','Plummer_Radius']

ValuePool = [[0.2],[0.01,0.05]]

In [ ]:
printU('All files in the directory will be overwritten. Proceed? [Y]')

if input() == 'Y':
    NRuns, save_options = PyUL.ParameterScanGenerator(baseConfig,ScanParams,
                            ValuePool,batch_path, 
                           KeepSmooth = False,
                            KeepResol = False,
                           SaveSpace = False)

In [ ]:
FullSim = True
T0 = time.time()
runs = os.listdir(batch_path)
runs.sort()

Run = 0

for run_folder in runs:
    
    if os.path.isdir(f'{batch_path}/{run_folder}'):
        
        RunText = f'Simulation #{Run+1} out of {NRuns}'
        PyUL.evolve(batch_path,run_folder,
            NBodyInterp = FullSim, SelfGravity = FullSim, NBodyGravity = True, 
            Silent = True, Shift = False, AutoStop = True, AutoStop2 = False, Message = RunText)

        Run +=1

        print(f'Simulation {Run} finished.')
        time.sleep(2)
        
import IPython
IPython.display.Audio("Done.mp3",autoplay=True)

print(f"{time.time()-T0:.3f}s")

# Analysis

This section is for Use with ** Dynamical Friction ** presets only.

## Energy Conservation Comparator

In [ ]:
### Initializing Journaling Variables

TLog  = [] # Time
RLog  = [] # Resol
EDLog = [] # Total Energy Change
aLog = [] # Smoothing Factor
KLog = [] # EK
EULog = [] # ULD E
SFLog = [] # Smoothing 
LLog = [] # Box Length

In [ ]:
batch_path = './Batches/' + PyUL.Runs('./Batches', Automatic = False)

runs = os.listdir(batch_path)
runs.sort()
Relative = True
Run = 0
for run_folder in runs:
    
    if os.path.isdir(f'{batch_path}/{run_folder}'):
        print(run_folder)
        TimeStamp = run_folder
        clear_output()
        loc = './' + batch_path + '/' + TimeStamp
        # Reload Config.. Why don't I use a class?
        NS, length, length_units, resol, duration, duration_units, step_factor, save_number, save_options, save_format, s_mass_unit, s_position_unit, s_velocity_unit, solitons,start_time, m_mass_unit, m_position_unit, m_velocity_unit, particles, embeds, Uniform,Density, density_unit ,a, UVel = PyUL.LoadConfig(loc)

        if save_number == -1:
            sim_number = PyUL.ULDStepEst(duration,duration_units,
                                                  length,length_units,
                                                  resol,step_factor, 
                                                  save_number = -1)
        else:
            sim_number = save_number

        # Dict of Run Data
        EndNum, Loaded = PyUL.Load_npys(loc,save_options)

        TMdata = Loaded['NBody']
        linedata = Loaded['1Density']
        graddata = Loaded['DF']

        print(Loaded['Directory'])
        Loaded = {}

        NBo = len(particles)


        print(f'SIM NUMBER IS {sim_number}')
        # Time
        durationMyr = CB(duration,duration_units,'Myr','t') * (EndNum-1) / sim_number

        # Space
        lengthKpc = CB(length,length_units,'kpc','l')
        lengthC = ToCode(length,length_units,'l')

        # Mass

        MassList_MSol = []
        MassListSI = []
        for TM in particles:
            MassList_MSol.append(CB(TM[0],m_mass_unit,'M_solar_masses','m'))
            MassListSI.append(CB(TM[0],m_mass_unit,'kg','m'))

        # Density
        DensitySI = CB(Density,density_unit,'kg/m3','d')
        DensityC = ToCode(Density,density_unit,'d')

        ## Misc. Pre-Multipliers


        # NBody_State Vector

        XPre = ToPhys(1,'kpc','l')
        VPre = ToPhys(1,'km/s','v')

        XPreSI = ToPhys(1,'m','l')
        VPreSI = ToPhys(1,'m/s','v')

        IArray = np.arange(len(TMdata[0]))

        # Dimensionful NBody State

        TMDataS = np.array(TMdata)
        TMDataSI = np.array(TMdata)

        TMDataS[:,IArray % 6 <= 2] *= XPre
        TMDataS[:,IArray % 6 >= 3] *= VPre


        TMDataSI[:,IArray % 6 <= 2] *= XPreSI
        TMDataSI[:,IArray % 6 >= 3] *= VPreSI


        # Rho data
        DPre = CB(1,density_unit,'kg/m3','d')
        # Energy
        EPre = PyUL.energy_unit
        # Field Strength
        PPre = PyUL.mass_unit / PyUL.length_unit

        # Translating Snapshots back to Time
        timerange = np.linspace(0,durationMyr,save_number+1)

        Tp = np.arange(EndNum)

        print(Tp)

        Tp = Tp * durationMyr / (sim_number+1)

        print(Tp)

        # Raw ULDM Energy

        # Total Energy from Integrator
        egylist = np.load('{}{}'.format(loc, '/Outputs/egylist.npy'),allow_pickle=True) * EPre

        # Energy Due to N Body Potential
        egpcmlist =  np.load('{}{}'.format(loc, '/Outputs/egpcmMlist.npy'),allow_pickle=True) * EPre # NEW
        egpcmlist2 = np.load('{}{}'.format(loc, '/Outputs/egpcmlist.npy'),allow_pickle=True) * EPre # OLD

        # Energy Due to ULDM Self-Interaction
        egpsilist = np.load('{}{}'.format(loc, '/Outputs/egpsilist.npy'),allow_pickle=True) * EPre

        # Energy Due to Quantum Fun
        ekandqlist = np.load('{}{}'.format(loc, '/Outputs/ekandqlist.npy'),allow_pickle=True) * EPre

        mtotlist = np.load('{}{}'.format(loc, '/Outputs/masseslist.npy'),allow_pickle=True) * EPre

        # Uniform Correction Performed in Integrator!

        # Reconstruct NBody Energy (Real)
        NBo, KS, PS = PyUL.NBodyEnergy(MassListSI,TMDataSI,EndNum, a, length_units)

        # Reconstruct NBody Energy (Physical)
        # NBo, KS, PS = PyUL.NBodyEnergy(MassListSI,TMDataSI,EndNum)

        if Relative:
            K0 = KS[0] #  Need attention
            EUnit = '$E_k(0)$'

            if UVel != [0,0,0]:

                VRelSI = CB(np.linalg.norm(UVel),s_velocity_unit,'m/s','v')

                printU(f'Initial Relative Speed is {VRelSI:.3f} m/s','QW')

                K0 = 1/2*MassListSI[0]*(VRelSI)**2

        else: 
            K0 = 1
            EUnit = 'J'
        # Processed ULDM Energy

        MES = (PS + KS)
        MESD = PyUL.GetRel(MES)/K0

        EKQD = PyUL.GetRel(ekandqlist)/K0
        EGPD = PyUL.GetRel(egpsilist)/K0

        ECMD = PyUL.GetRel(egpcmlist)/K0
        ECOD = PyUL.GetRel(egpcmlist2)/K0

        KSD = PyUL.GetRel(KS)/K0
        PSD = PyUL.GetRel(PS)/K0

        EUOld  = egylist
        EUOldD = PyUL.GetRel(EUOld)/K0

        EUNew  = egpsilist + ekandqlist + egpcmlist2
        EUNewD = PyUL.GetRel(EUNew)/K0

        ETOld  = EUOld + MES
        ETOldD = PyUL.GetRel(ETOld)/K0

        ETNew  = EUNew + MES
        ETNewD = PyUL.GetRel(ETNew)/K0


        EROld = ETOld / ETOld[0]

        ERNew = ETNew / ETNew[0]
        LLog.append(lengthKpc)
        RLog.append(resol)
        TLog.append(Tp)
        EDLog.append(ERNew)
        KLog.append(MESD)
        EULog.append(EUNewD)
        SFLog.append(step_factor)   
        
        rP = PyUL.RecPlummer(a,length_units)
        aLog.append(rP)
        clear_output()
printU('Done!','IO')

In [ ]:
fig = plt.figure(figsize = (10,8), dpi = 300)

ax = fig.add_subplot(111)

plt.subplots_adjust(wspace=0, hspace=0)

j = -1
for i in zip(TLog,RLog,EDLog,EULog,KLog,aLog,SFLog,LLog):
    j += 1

    ax.plot(i[0],i[2],label = f'N={i[1]}$^3$, $r_{{Plummer}}$={i[5]:.3g} pc')



#ax.set_yticks([-0.01,0.,0.01])
#ax.set_yticklabels(['$-$1‱','0‱','1‱'])

ax.set_xlabel('Time / Myr')
ax.set_ylabel('$E_{Total}$ / $ E_{Total}(0)$')


plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

#plt.ylim([0.999998,1.000002])


ax.legend(ncol =2 ,loc = 8, fontsize = 11)

plt.show()

In [ ]:
fig = plt.figure(figsize = (10,6.18), dpi = 300)
ax = fig.add_subplot(111)

EColors = plt.cm.plasma(np.linspace(0,1,25))
j = 0
for i in zip(TLog,RLog,EDLog,EULog,KLog,aLog,SFLog,LLog):


    j += 1
    ax.plot(i[0],i[4],'.-',label = f'N={i[1]}$^3$, $r_{{Plummer}}$={i[5]:.3g} pc', color = EColors[j])

ax.plot([0,32],[-1,-1], 'k--')    
ax.plot([0,32],[1,1], 'k--') 

ax.set_ylabel(f'Change in Energy / {EUnit}')
plt.xlabel('Time / Myr')
plt.xlim(0,32)
plt.legend(fontsize = 8, ncol = 2)

## Single Run Analyzer

This section is for Use with ** Dynamical Friction ** presets only.

In [26]:
batch_path = './Batches/' + PyUL.Runs('./Batches', Automatic = False)

# save_path = batch_path
TimeStamp = PyUL.Runs(batch_path,Automatic = False)
clear_output()
loc = './' + batch_path + '/' + TimeStamp
# Reload Config.. Why don't I use a class?
NS, length, length_units, resol, duration, duration_units, step_factor, save_number, save_options, save_format, s_mass_unit, s_position_unit, s_velocity_unit, solitons,start_time, m_mass_unit, m_position_unit, m_velocity_unit, particles, embeds, Uniform,Density, density_unit ,a, UVel = PyUL.LoadConfig(loc)

DLog = []
TLog = []
CLog = []
MLog = []
YLog = []
ELog = []
XLog = []

if save_number == -1:
    sim_number = PyUL.ULDStepEst(duration,duration_units,
                                          length,length_units,
                                          resol,step_factor, 
                                          save_number = -1)
else:
    sim_number = save_number
    
# Dict of Run Data
EndNum, Loaded = PyUL.Load_npys(loc,save_options)
TMdata = Loaded['NBody']
linedata = Loaded['1Density']
graddata = Loaded['DF']

print(Loaded['Directory'])
Loaded = {}

NBo = len(particles)

# Time
durationMyr = CB(duration,duration_units,'Myr','t') * (EndNum-1) / sim_number

# Space
lengthKpc = CB(length,length_units,'kpc','l')
lengthC = ToCode(length,length_units,'l')

# Mass

MassList_MSol = []
MassListSI = []
for TM in particles:
    MassList_MSol.append(CB(TM[0],m_mass_unit,'M_solar_masses','m'))
    MassListSI.append(CB(TM[0],m_mass_unit,'kg','m'))
    
# Density
DensitySI = CB(Density,density_unit,'kg/m3','d')
DensityC = ToCode(Density,density_unit,'d')

YLog.append(DensityC)

## Misc. Pre-Multipliers

Tp = np.linspace(0,EndNum-1,EndNum)
Tp = Tp * durationMyr / (sim_number+1)

# NBody_State Vector

XPre = ToPhys(1,'kpc','l')
VPre = ToPhys(1,'km/s','v')

XPreSI = ToPhys(1,'m','l')
VPreSI = ToPhys(1,'m/s','v')

IArray = np.arange(len(TMdata[0]))

# Dimensionful NBody State

TMDataS = np.array(TMdata)
TMDataSI = np.array(TMdata)

TMDataS[:,IArray % 6 <= 2] *= XPre
TMDataS[:,IArray % 6 >= 3] *= VPre


TMDataSI[:,IArray % 6 <= 2] *= XPreSI
TMDataSI[:,IArray % 6 >= 3] *= VPreSI


DLog.append(linedata)
MLog.append(TMDataS)

DPre = CB(1,density_unit,'kg/m3','d')
# DYNFRIC

vRel0 = -CB(UVel[1],s_velocity_unit,'m/s','v') + TMDataSI[:,4]

mBH = MassList_MSol[0] # Million MSols
mBH_kg = CB(mBH,'M_solar_masses','kg','m')

lR = PyUL.hbar/(PyUL.axion_mass*vRel0) # Has a factor of 2pi removed.

lRP = PyUL.convert_between(lR,'m','pc','l')

# Quantum Mach Number

MQ = 44.56*vRel0/1000*(((1e-22*PyUL.eV))/PyUL.axion_mass)/mBH/10

beta = 2*np.pi*PyUL.G*mBH_kg/(vRel0[0])**2/(lR[0])

# Reference Force Value

FRel = 4*np.pi*Density*DPre*(PyUL.G*mBH*1e6*PyUL.solar_mass/(vRel0))**2

# Time in seconds
TpSec = CB(Tp,'Myr','s','t')

# Motion due to Particle Movement
dSim = TMDataSI[:,1] - TMDataSI[0,1]

vQW = -1*CB(UVel[1],s_velocity_unit,'m/s','v')

dQW = vQW*TpSec

bSI = dQW + dSim

bV = CB(bSI,'m','pc','l')

BVTilde = bV/lRP

Lambda = BVTilde/beta

# Expected DynDrag

CLambda = np.log(2*Lambda) + 1/Lambda*np.log(2*Lambda) - 1


# Gravitational Attraction of DM Wake
CLogDF2 = -1*np.array(graddata)[:,1] * PyUL.length_unit/PyUL.time_unit**2 # 1 Code Acceleration Unit

# Phi is already in m/s^2 

Crel = CLogDF2*vRel0**2/(4*np.pi*CB(Density,density_unit,'kg/m3','d')*PyUL.G**2*mBH_kg)

PlotRange = np.linspace(-lengthKpc/2, lengthKpc/2,resol,endpoint = False)
# Exact Evaluation

from scipy.special import sici as SiCin

SbV,CbV = SiCin(2*BVTilde)

RealCbV = np.euler_gamma + np.log(2*BVTilde) - CbV

CrelV = RealCbV+np.sin(2*BVTilde)/(2*BVTilde)-1

CrelMax = np.max(Crel)

TLog.append(BVTilde)
CLog.append(Crel)

ELog.append(EndNum)

XLog.append(PlotRange)

Loaded = {}

The required number of ULDM steps is 434
The actual ULDM steps is 434
PyUL2.LoadNPY: Loaded 435 Data Entries from ././Batches/Pub1_ERS_2/PScan_F01_A01
././Batches/Pub1_ERS_2/PScan_F01_A01


### The 1-Dimensional Slice

In [ ]:
YLog

In [ ]:
Step = 34
VOffset = 0.03
Sotal = EndNum // Step
Cutoff = 167


NSamples = len(DLog)

C1 = 'goldenrod'
C2 = 'darkolivegreen'
C3 = 'plum'
C4 = 'cadetblue'


Cs = [C1,C2,C3,C4]


fig = plt.figure(figsize = (16,9))

ax = fig.add_subplot(111)

plt.setp(ax.spines.values(), linewidth=1)

for i in range(Cutoff):
    if i%Step == 0:
        Alpha = 0.5
        LW = 2
        
        #plt.plot(PlotRange,0*PlotRange+1+VOffset*i,'--',alpha = 1, color = 'k', lw = 1)
        
        #plt.plot(PlotRange,DLog[0][i]/DensityC+VOffset*i,'-',alpha = Alpha, color = C1, lw = 3)
        #plt.plot(PlotRange,DLog[1][i]/DensityC+VOffset*i,'-',alpha = Alpha, color = C2, lw = 3)

            
        for J in range(NSamples):
            
            j = -J
            
            print(len(XLog[j]),len(DLog[j][i]))
            
            try:
                plt.fill_between(XLog[j],DLog[j][i]/YLog[j]+VOffset*i,1+VOffset*i,alpha = Alpha, color = Cs[J], 
                                 lw = LW, zorder = 100+j*10)
                
                plt.scatter([MLog[j][i,1]],[1+VOffset*i],c = Cs[J], alpha = 1, zorder = 300 + 20*J)

            except IndexError:
                print(f'Branch #{j} ran out of data points ({ENum[j]} vs. {i}).')
                continue

for j in range(NSamples):
    
    Cutoff = np.min([Cutoff,ELog[j]])
    
    #plt.plot(MLog[j][:Cutoff,1],np.arange(Cutoff)*VOffset+1,'-.', color = Cs[j], lw = 2, alpha = 0.5)
            
plt.ylabel(r'Time $\longrightarrow$')
plt.xlabel(r'x / kpc')
plt.yticks([])
#plt.xlim(0.,7.5)

### Dynamical Friction C-b

In [ ]:
plt.figure(figsize = (10,8))

BLine = np.linspace(-2,3,300)

BLineX = 10**BLine

rSys = PyUL.convert_between(length*BLineX,length_units,'m','l')

Lambda = vRel0[0]**2*rSys/(PyUL.G*mBH_kg)

# Classical B

C_Cl = np.log(2*Lambda) - 1 + 1/Lambda*np.log(2*Lambda)

F_Cl = 4*np.pi*PyUL.G**2*mBH_kg**2*DensityC*PyUL.convert_back(1,'kg/m3','d')/(vRel0[0]**2)*C_Cl / FRel[0]

# Classical A

C_A = (1+Lambda)/(Lambda)*np.arctanh(np.sqrt(Lambda**2+2*Lambda)/(1+Lambda)) - np.sqrt(1+2/Lambda)


plt.loglog(BLineX,C_A,'--',
           color = EmphColor, label = 'Classical Dynamical Friction Limit')

#plt.loglog(BLineX,C_Cl,'--',label = 'Classical Dynamical Friction Limit B')

from scipy.special import sici as SiCin

Sb,Cb = SiCin(2*BLineX)

RealCb = np.euler_gamma + np.log(2*BLineX) - Cb

plt.loglog(BLineX,RealCb+np.sin(2*BLineX)/(2*BLineX)-1,
           color = EmphColor, label = 'Coulomb Scattering Result')

#plt.loglog(BLineX,1/3*BLineX**2,color = 'green',label = 'Lancaster Low b Asymptote')

from scipy.special import digamma as Dig

beta = PyUL.G*mBH_kg/(vRel0[0]**2*lR[0])


plt.scatter(TLog[0],CLog[0],label = 'Simulation' ,  color = C1, s = 35, marker = 'x')
#plt.scatter(TLog[1],CLog[1],label = 'Full Simulation' ,  color = C2, s = 35, marker = '1')
#plt.fill_betweenx(Crel,BCutTildeMin, BCutTildeMax,  color = 'r', alpha = 0.2)


plt.xticks([0.1,1,10,100,1000])
plt.yticks([0.1,1,10])

plt.xlabel('$\~{b} $')
plt.ylabel('$C$ ')

plt.ylim(0.005,300)

#plt.xlim(0.05,200)

plt.legend(ncol = 2, fontsize = 12)